# Image matching dataset
- Download the dataset from Kaggle competition page: [Image Matching Challenge 2025](https://www.kaggle.com/competitions/image-matching-challenge-2025/data)
- Custom Dataset class to load the dataset
- Normalize the images with mean and std of 0.5 for all channels.

In [8]:
from data_preprocess.image_matching_dataset import ImageMatchingDataset
import torchvision.transforms as transforms
import torch

train_dataset = ImageMatchingDataset(labels_path='data/train_labels.csv', root_dir='data/train',
                               transform=transforms.Compose([
                                   transforms.ToTensor(),
                                   transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
                               ]))

## Initial Analysis of the dataset

## Creating descriptors for the dataset and saving them

In [13]:
from torch.utils.data import DataLoader

DEVICE = torch.device(0 if torch.cuda.is_available() else -1)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=10)


In [ ]:
'''Replace extract_descriptor with proper function'''
import datetime
import os
BATCH_SIZE = 16
NUM_WORKERS = 1
dir_name = 'data_descriptors'

processed_data = []

for batch in train_loader:
    descriptors = compute_descriptors(batch)
    batch['sift_descriptors'] = descriptors
    processed_data.append(batch)



print('-----Saving descriptors-----')

if not os.path.exists(dir_name):
    os.makedirs(os.path.dirname(dir_name), exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
torch.save(processed_data, os.path.join(dir_name, f'image_matching_{timestamp}.pt'))





In [ ]:
def process_single_image(image):
    """Function to process single image with SIFT - will run in parallel"""
    # Create SIFT detector (each worker needs its own instance)
    sift = cv2.SIFT_create(nfeatures=1000)
    
    # Convert tensor to numpy for OpenCV
    if torch.is_tensor(image):
        image = image.permute(1, 2, 0).numpy()
    
    # Convert RGB to BGR for OpenCV
    image_bgr = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    # Compute SIFT
    keypoints, descriptors = sift.detectAndCompute(image_bgr, None)
    
    return keypoints, descriptors

# Custom collate function to handle SIFT processing in parallel
def sift_collate_fn(batch):
    """Process batch in parallel using DataLoader workers"""
    images = [item['image'] for item in batch]
    
    # Process each image in the batch (this runs in parallel across workers)
    sift_results = [process_single_image(img) for img in images]
    
    # Add SIFT results back to batch items
    for item, (kp, desc) in zip(batch, sift_results):
        item['keypoints'] = kp
        item['descriptors'] = desc
    
    return batch

# Create DataLoader with parallel processing
dataloader = DataLoader(
    dataset,
    batch_size=32,  # Now we can use larger batch size
    num_workers=4,  # Number of parallel workers
    shuffle=False,
    collate_fn=sift_collate_fn  # Use our custom collate function
)

# Process and save data
processed_data = []
for batch in dataloader:
    processed_data.extend(batch)  # batch is already processed by collate_fn

# Save the processed data
torch.save(processed_data, 'path/to/save/sift_features.pt')